In [ ]:
!pip install --quiet openai python-dotenv langchain pypdf tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2202684 sha256=be2cb5d6c4a120430fb5b55cc784e1273c787d2ff5b9ecc7e67d695e1c6f3425
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hnswlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Chat

Recall the overall workflow for retrieval augmented generation (RAG):

In [ ]:
# ojo cargar el documetno .env con las api key de todas las librerias
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023.
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [ ]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


### Para chroma
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'docs/chroma/'

embedding = OpenAIEmbeddings()

vectordb = Chroma(persist_directory=persist_directory,

embedding_function=embedding)

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_001_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_002_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_003_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_008_NG_Dinarp_2023.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

102

In [ ]:
embedding = OpenAIEmbeddings()

vectordb = FAISS.from_documents(
    splits,
    embedding
)

### Similarity Search

In [ ]:
question = "¿Qué se resuelve en el documento RESOLUCIÓN No. 002-NG-DINARP-2023?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [ ]:
docs[0]

Document(page_content='Excepcionales de Datos e Información de las Entidades que Conforman el SINARDAP.  \n \nLa presente resolución entrará en vigencia a partir de la fecha de su suscripción, sin \nperjuicio de su publicación en el R egistro Oficial.  \n \nDado en la ciudad de Quito, a los 12 días del mes de mayo de 2023.  \n \n \n \n \n \n \n \nAbg. Daniel Augusto Arboleda Villacreses  \nDIRECTOR NACIONAL (E)  \nDIRECCIÓN  NACIONAL DE REGISTROS PÚBLICOS', metadata={'source': '/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_003_NG_Dinarp_2023.pdf', 'page': 16})

In [ ]:
docs[1]

Document(page_content='“Finalmente, pongo en su conocimiento el proyecto de resolución de la mentada \nresolución para su  revisión y posterior suscripción, en virtud de las \nrecomendaciones de los informes adjuntos y del análisis de esta u nidad, por lo \nque recomiendo su aprobación ”;  \n \nQue a través de los resultados obtenidos por medio del formulario remitido durante la \nplenaria dos del  evento “Foro  Nacional d e Registros - Primera edición” realizado \nlos días 20 y 21 de julio de 2023, se dete rminó el tiempo prudente para la \ntransformación digital de los registros y la priorización al momento de digitalizar.  \n \nQue en ejercicio de las facultades dispuestas en  la Ley de Transformación Digital y \nAudiovisual  es necesario establecer las característic as y condiciones que  deben \ncumplir los sistemas informáticos que permitan la transformación a formato digital \nde todos los registros ; y, \n \nQue con Acuerdo Ministerial Nro. MINTEL -MINTEL -2023 -0004, de 06 de a

In [ ]:
docs[2]

Document(page_content='Página 13 de 16 \n    \n \n \n \n \nimplementación.  Para el efecto, los registros priorizarán los últimos veinticinco años de su \narchivo.  \nDISPOSICIONES DEROGATORIAS  \nPRIMERA: Deróguese la R esolución Nro. 019 -NG-DINARDAP -2013  de 9 de m ayo de \n2013, que contiene la norma que “Crea el Sistema Nacional de Registro de la \nPropiedad ”.  \nSEGUNDA: Deróguese toda norma que se encuentre en contraposición a la presente \nresolución.  \nDado en la ciudad de Quito, a los 31 días del mes de jul io de 2023.  \n \n \n \nAbg. Daniel Augusto Arboleda Villacreses  \nDIRECTOR NACIONAL (E)  \nDIRECCIÓN NACIONAL DE REGISTROS PÚBLICOS  \n \n \n \n \n Aprobado:  Ab. Geanella Pincay Palacios / \nCoordinadora de Normativa y de Protección \nde la Información   \nRevisado:  Ab. Sofía Vázquez / Directora de No rmativa   \nElaborado:  Ab. / Carolina Morales / Asistente', metadata={'source': '/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_008_NG_Dinarp_

### ***************************************************************** ________________________________________________________________________________________________________________________________________________________________


In [ ]:
question = "¿Por qué se incorpora al Ministerio de Defensa Nacional como parate del Sistema de Registros Públicos?"
docs = vectordb.similarity_search(question,k=4)
len(docs)

4

In [ ]:
docs[0]

Document(page_content='Registros Públicos y su Reglamento de aplicación,  \n \nRESUELVE:  \n \nArtículo 1. - Incorporar al “Ministerio de Defensa Nacional” como parte del Sistema \nNacional de Registros Públicos , por ser una institución que posee y administra información \npública perteneciente a personas naturales.  \n \nArtículo 2. - Integrar los datos otorgados por e l “Ministerio de D efensa Nacional” al \nSistema Nacional de Registros Públicos , conforme a l siguiente detalle:', metadata={'source': '/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_002_NG_Dinarp_2023.pdf', 'page': 3})

In [ ]:
docs[1]

Document(page_content='justificación jurídica, y determinar una finalidad clara de su uso bajo el amparo de las \ndisposiciones de la Ley Orgánica de Protección de Datos Personales . (…)”;  \n \nQue,  mediante memorando s Nros. DINARP -DGR -2023-0017 -M de 23 de enero de \n2023 y DINARP -DGR -2023 -0020 -M de 30 de enero de 2023, suscritos por la \nDirección de Gestión y Registro,  parafraseando el contenido de los mismos, \ninforma: Conforme a lo establecido en el Art. 4 del Reglamento de la Ley del Sistema Nacional \nde Registros de Datos Públicos, el Ministerio de Defensa Nacional mantiene una base  relacionada \na personas naturales y es administrada por una entidad del sector público; por lo cual me permito \nseñala r el cumplimiento de la normativa referente a la incorporación de los entes registrales al \nSistema Nacional de Registros Públicos.  \n \nQue,  mediante Acuerdo Ministerial Nro. MINTEL -MINTEL -2021 -0003 de 22 de junio \ndel 2021, la doctora Vianna Maino Isaías, Min 

In [ ]:
docs[2]

Document(page_content='de Gestión y Registro, parafrasea ndo el contenido de los mismos, informa:  Conforme \na lo establecido en el Art. 4 del Reglamento de la Ley del Sistema Nacional  de Registros de Datos \nPúblicos y la normativa enunciada por el Instituto Nacional de Estadísticas y Censos, misma q ue \nhace referencia a información relacionada a personas naturales  y es administrada por una entidad \ndel sector público; me permito señalar e l cumplimiento de la normativa referente a la incorporación \nde los entes registrales al Sistema Nacional de Registros Públicos. ”  \n \nQue,  mediante Acuerdo Ministerial Nro. MINTEL -MINTEL -2021 -0003 de 22 de junio \ndel 2021, la doctora Vianna Maino Isaías, Mini stra de Telecomunicaciones y de la \nSociedad de la Información, resuelve: “(...) Designar a la abogada Angie Karina Jijón \nMancheno como Directora Nacional de Registros Públicos, quién ejercerá las competencias y \natribuciones previstas en la Le y Orgánica de l Sistema Naciona

In [ ]:
docs[3]

Document(page_content='Página 2 de 6 \n    \n \n \n \n \npara acceder a la información sobre el patrimonio de las personas cualquier solicitante deberá \njustificar y motivar su requerimiento, declarar el uso que hará del mismo y consignar sus datos \nbásicos de identidad, tal es como nombres y apellidos completos, número del documento de identidad \no ciudadanía, dirección domiciliaria y los demás datos que mediante el respectivo reglamento se \ndeterminen. Un uso distinto al declarado dará lugar a la determinación de responsabilid ades, sin \nperjuicio de las acciones legales que el titular de la información pueda ejercer. La Directora o \nDirector Nacional de Registros Públicos, definirá los demás datos que integran el sistema nacional \ny el tipo de reserva y accesibilidad .”;  \n \nQue,   el artículo 31 de la Ley Orgánica del Sistema Nacional de Registros Públicos, \ndetermina entre otras, las siguientes atribuciones y facultades de la Dirección \nNacional de Registros Públicos: “

In [ ]:
#prueba simple definición del llm

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("¡Hola Mundo!")

'¡Hola! ¿En qué puedo ayudarte hoy?'

### Retrieval QA

In [ ]:
### Retrieval QA
# Build prompt
from langchain.prompts import PromptTemplate
template = """Usa la siguiente pieza de contexto para responder la pregunta al final. Si no sabes la respuesta solo di, "Yo no se", no trates de inventar una respuesta. Usa tres frases máximo. Mantén la respuesta concisa. Siempre di: "Gracias por preguntar", al final de la respuesta.
{context}
Pregunta: {question}
Respuesta útil:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [ ]:
# Run chain
from langchain.chains import RetrievalQA
question = "Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'El artículo 86 de la Ley de Transformación Digital y Audiovisual señala que los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, adaptando todos los procedimientos contemplados en la ley para los registros físicos de los documentos. Gracias por preguntar.'

In [ ]:
question = "¿Por qué se incorpora al Ministerio de Defensa Nacional como parate del Sistema de Registros Públicos?"
result = qa_chain({"query": question})
result["result"]

'El Ministerio de Defensa Nacional se incorpora al Sistema Nacional de Registros Públicos porque posee y administra información pública perteneciente a personas naturales. Gracias por preguntar.'

In [ ]:
question = "¿De qué es responsable el Ministerio de Defensa Nacional como parate del Sistema de Registros Públicos?"
result = qa_chain({"query": question})
result["result"]

'El Ministerio de Defensa Nacional es responsable de poseer y administrar información pública perteneciente a personas naturales como parte del Sistema Nacional de Registros Públicos. Gracias por preguntar.'

In [ ]:
question = "¿Qué determina el artículo 31 de la Ley del Sistema Nacional de Registros Públicos?"
result = qa_chain({"query": question})
result["result"]

'El artículo 31 de la Ley Orgánica del Sistema Nacional de Registros Públicos determina las atribuciones y facultades de la Dirección Nacional de Registros Públicos, como presidir el sistema, dictar resoluciones y normas necesarias, y consolidar y administrar la base única de datos de todos los registros públicos. Gracias por preguntar.'

In [ ]:
question = "¿Quién es la DIRECTORA NACIONAL DE REGISTROS PÚBLICOS?"
result = qa_chain({"query": question})
result["result"]

'La Directora Nacional de Registros Públicos es la abogada Angie Karina Jijón Mancheno. Gracias por preguntar.'

### Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?"
result = qa({"question": question})

In [ ]:
result['answer']

'El artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro señala: "Agréguese a continuación del artículo 24, en el TÍTULO V, el siguiente artículo innumerado: \'Art. (…). - Los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, para lo cual se tomarán en consideración y serán adaptados todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos\'".'

In [ ]:
result

{'question': '¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?',
 'chat_history': [HumanMessage(content='¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?'),
  AIMessage(content='El artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro señala: "Agréguese a continuación del artículo 24, en el TÍTULO V, el siguiente artículo innumerado: \'Art. (…). - Los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, para lo cual se tomarán en consideración y serán adaptados todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos\'".')],
 'answer': 'El artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro señala: "Agréguese a continuación del artículo 24, en el TÍTULO V, el siguiente artículo innumerado: \'Art. (…). - Los registros de las inscripciones y el libro de índice gen

In [ ]:
question = "¿Cómo podrán llevarse los registros según el artículo 86?"
result = qa({"question": question})

In [ ]:
result['answer']

'Según el artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro, los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica. Para ello, se tomarán en consideración y se adaptarán todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos.'

In [ ]:
result

{'question': '¿Cómo podrán llevarse los registros según el artículo 86?',
 'chat_history': [HumanMessage(content='¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?'),
  AIMessage(content='El artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro señala: "Agréguese a continuación del artículo 24, en el TÍTULO V, el siguiente artículo innumerado: \'Art. (…). - Los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, para lo cual se tomarán en consideración y serán adaptados todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos\'".'),
  HumanMessage(content='¿Cómo podrán llevarse los registros según el artículo 86?'),
  AIMessage(content='Según el artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro, los registros de las inscripciones y el libro de índice general podrán llevarse de forma elect

In [ ]:
question = "¿Por qué se incorpora al Ministerio de Defensa Nacional como parate del Sistema de Registros Públicos?"
result = qa({"question": question})
result['answer']

'La razón de incorporar al Ministerio de Defensa Nacional como parte del Sistema Nacional de Registros Públicos es porque el Ministerio de Defensa Nacional posee y administra información pública perteneciente a personas naturales.'

In [ ]:
question = "¿De qué es responsable el Ministerio de Defensa Nacional como parate del Sistema de Registros Públicos?"
result = qa({"question": question})
result['answer']

'La responsabilidad del Ministerio de Defensa Nacional como parte del Sistema Nacional de Registros Públicos es poseer y administrar información pública perteneciente a personas naturales.'

In [ ]:
question = "¿Quién es la DIRECTORA NACIONAL DE REGISTROS PÚBLICOS?"
result = qa({"question": question})
result['answer']

'La directora nacional de registros públicos es Angie Karina Jijón Mancheno.'

# Create a chatbot that works on your documents

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

The GUI appearance also varies depending on the platform it is running on.

In [ ]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa


In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_008_NG_Dinarp_2023.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F5'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F5'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F5'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F5'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F5'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return


### Create a chatbot

In [ ]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.JPG( './img/convchain.jpg') #JPG instead of Image

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] JPG(str, width=400)

Feel free to copy this code and modify it to add your own features. You can try alternate memory and retriever models by changing the configuration in load_db function and the convchain method. Panel and Param have many useful features and widgets you can use to extend the GUI.

## Acknowledgments

Panel based chatbot inspired by Sophia Yang, [github](https://github.com/sophiamyang/tutorials-LangChain)